# Federated Learning for Posture Classification

This notebook provides a comprehensive walkthrough of a federated learning system designed to classify posture data while preserving privacy across multiple clients. The system uses PyTorch Lightning and implements the FedAvg (Federated Averaging) algorithm with advanced data augmentation techniques.

## 🎯 Project Overview

**Goal**: Train a posture classification model across multiple clients without sharing raw data, maintaining privacy while achieving good performance.

**Key Features**:
- Federated learning with IID and Non-IID data distributions
- Advanced data augmentation with SMOTE and noise injection
- Real-time TensorBoard logging and visualization
- Comprehensive evaluation metrics

## 📊 Dataset Structure

The project works with posture data containing 4 key features:
- `neck_angle`: Angle of the neck relative to vertical
- `torso_angle`: Angle of the torso relative to vertical
- `shoulders_offset`: Horizontal offset between shoulders
- `relative_neck_angle`: Neck angle relative to torso

**Target**: Binary classification (0: Bad Posture, 1: Good Posture)

---

## 🏗️ Architecture Overview

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐ 
│ FederatedServer │    │ FederatedTrainer│    │ FederatedClient │ 
│                 │    │                 │    │                 │ 
│ • Global Model  │    │ • Orchestrates  │    │ • Local Training│ 
│ • Weight Aggreg.│    │ • Logging       │    │ • Local Eval    │ 
│ • Evaluation    │    │ • Checkpointing │    │ • Data Privacy  │ 
└─────────────────┘    └─────────────────┘    └─────────────────┘ 
         │                       │                       │        
         └───────────────────────┼───────────────────────┘        
                                 │                                
                        ┌─────────────────┐                       
                        │ PostureMLP Model│                       
                        │                 │                       
                        │ 4 → 64 → 32 → 2 │                       
                        │ Dropout         │                       
                        │ ReLU Activation │                       
                        └─────────────────┘                       
```
---

## 🧠 Neural Network Model

### 🔍 Model Architecture Explanation

The `PostureMLP` is a PyTorch Lightning module with the following key components:

1. **Input Layer**: 4 features (neck angle, torso angle, shoulders offset, relative neck angle)
2. **Hidden Layers**: 64 → 32 neurons with ReLU activation and dropout (0.2) for regularization
3. **Output Layer**: 2 neurons for binary classification (good/bad posture)
4. **Loss Function**: CrossEntropyLoss for multi-class classification
5. **Metrics**: Accuracy tracking for train/validation/test phases
6. **Visualization**: Automatic confusion matrix and feature distribution logging


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as pl
import matplotlib.pyplot as plt
import seaborn as sns
from torchmetrics import Accuracy, ConfusionMatrix


class PostureMLP(pl.LightningModule):
    def __init__(self, learning_rate=0.001):
        super().__init__()

        # Save hyperparameters
        self.save_hyperparameters()

        # Define the MLP architecture
        self.fc1 = nn.Linear(4, 64)  # 4 input features
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # 2 classes for binary classification
        self.dropout = nn.Dropout(0.2)

        # Define loss function
        self.loss_fn = nn.CrossEntropyLoss()

        # Define metrics for tracking accuracy
        self.train_accuracy = Accuracy(task="binary")
        self.val_accuracy = Accuracy(task="binary")
        self.test_accuracy = Accuracy(task="binary")

        # Confusion matrix for test evaluation
        self.confusion_matrix = ConfusionMatrix(task="binary")

        # Store learning rate
        self.learning_rate = learning_rate

        # Class names for visualization
        self.class_names = ["Bad Posture", "Good Posture"]

    def forward(self, x):
        # Define the forward pass
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

    def training_step(self, batch, batch_idx):
        # This defines what happens in one training step
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = self.train_accuracy(preds, y)

        # Log metrics
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)

        # Log histograms every 100 steps
        if batch_idx % 100 == 0:
            for name, param in self.named_parameters():
                if param.grad is not None:
                    self.logger.experiment.add_histogram(
                        f"weights/{name}", param, self.global_step
                    )
                    self.logger.experiment.add_histogram(
                        f"gradients/{name}", param.grad, self.global_step
                    )

        return loss

    def validation_step(self, batch, batch_idx):
        # This defines what happens in one validation step
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = self.val_accuracy(preds, y)

        # Log metrics
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        # This defines what happens in one test step
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = self.test_accuracy(preds, y)

        # Update confusion matrix
        self.confusion_matrix.update(preds, y)

        # Log metrics
        self.log("test_loss", loss)
        self.log("test_acc", acc)

        return loss

    def on_validation_epoch_end(self):
        # Log feature distributions every 5 epochs
        if self.current_epoch % 5 == 0:
            try:
                # Get validation dataloader
                val_dataloader = self.trainer.datamodule.val_dataloader()

                # Get a batch of validation data
                batch = next(iter(val_dataloader))
                features, labels = batch
                features = features[:100]  # Take first 100 samples
                labels = labels[:100]

                # Move to device
                features = features.to(self.device)

                # Get predictions
                with torch.no_grad():
                    logits = self(features)
                    preds = torch.argmax(logits, dim=1)

                # Create feature distribution plot
                fig, axes = plt.subplots(2, 2, figsize=(12, 10))
                feature_names = [
                    "Neck Angle",
                    "Torso Angle",
                    "Shoulders Offset",
                    "Relative Neck Angle",
                ]

                for i, (ax, feature_name) in enumerate(zip(axes.flat, feature_names)):
                    good_posture_mask = labels == 1
                    bad_posture_mask = labels == 0

                    ax.hist(
                        features[good_posture_mask, i].cpu().numpy(),
                        alpha=0.7,
                        label="Good Posture",
                        bins=20,
                        color="green",
                    )
                    ax.hist(
                        features[bad_posture_mask, i].cpu().numpy(),
                        alpha=0.7,
                        label="Bad Posture",
                        bins=20,
                        color="red",
                    )
                    ax.set_title(f"{feature_name} Distribution")
                    ax.legend()
                    ax.grid(True, alpha=0.3)

                plt.tight_layout()
                plt.suptitle(
                    f"Feature Distributions - Epoch {self.current_epoch}", y=1.02
                )

                # Log to tensorboard
                self.logger.experiment.add_figure(
                    "feature_distributions", fig, self.current_epoch
                )
                plt.close(fig)

            except Exception as e:
                print(f"Could not log feature distributions: {e}")

    def on_test_epoch_end(self):
        # Compute and log confusion matrix
        cm = self.confusion_matrix.compute()

        # Create matplotlib figure
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.heatmap(
            cm.cpu().numpy(),
            annot=True,
            fmt="d",
            ax=ax,
            xticklabels=self.class_names,
            yticklabels=self.class_names,
        )
        ax.set_xlabel("Predicted")
        ax.set_ylabel("Actual")
        ax.set_title("Confusion Matrix")
        plt.tight_layout()

        # Log to tensorboard
        self.logger.experiment.add_figure("confusion_matrix", fig, self.current_epoch)
        plt.close(fig)

    def configure_optimizers(self):
        # Define the optimizer
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def on_train_start(self):
        """Log model graph when training starts"""
        try:
            # Get a sample from the training dataloader
            sample_batch = next(iter(self.trainer.datamodule.train_dataloader()))
            sample_input = sample_batch[0][:1]  # Take just one sample

            # Move to same device as model
            sample_input = sample_input.to(self.device)

            # Log the model graph
            self.logger.experiment.add_graph(self, sample_input)
            print("Model graph logged to TensorBoard")

        except Exception as e:
            print(f"Could not log model graph: {e}")

## 🗄️ Data Module with Advanced Augmentation

### 🎲 Data Augmentation Strategy

The augmentation pipeline includes multiple sophisticated techniques:

1. **Gaussian Noise**: Simulates sensor measurement noise
2. **Angle Variations**: Small random angle changes (±2°) to simulate natural movement
3. **Correlated Noise**: Realistic correlation between neck and torso angles
4. **SMOTE**: Synthetic Minority Oversampling Technique for balanced data generation
5. **Regularization Noise**: Creates "hard" examples to improve model robustness

In [ ]:
from typing import List

import lightning as pl
import numpy as np
import pandas as pd
import torch
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader, TensorDataset


class AugmentedPostureDataset(Dataset):
    """Custom dataset with real-time augmentation"""

    def __init__(
            self, features, labels, augment=True, noise_std=0.05, augment_prob=0.5
    ):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)
        self.augment = augment
        self.noise_std = noise_std
        self.augment_prob = augment_prob

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        x = self.features[idx].clone()
        y = self.labels[idx]

        if self.augment and torch.rand(1) < self.augment_prob:
            x = self._augment_sample(x)

        return x, y

    def _augment_sample(self, x):
        """Apply augmentation to a single sample"""
        augmented = x.clone()

        # 1. Add Gaussian noise to simulate sensor noise
        noise = torch.normal(0, self.noise_std, size=x.shape)
        augmented += noise

        # 2. Small angle variations (±2 degrees converted to your scale)
        angle_noise = torch.normal(0, 0.02, size=x.shape)
        augmented += angle_noise

        # 3. Simulate slight measurement inconsistencies
        # Add correlated noise between neck and torso angles (they're related)
        if len(x) >= 2:  # neck_angle and torso_angle
            correlation_noise = torch.normal(0, 0.01, size=(1,)).item()  # Get scalar value
            augmented[0] += correlation_noise  # neck_angle
            augmented[1] += correlation_noise * 0.5  # torso_angle (less correlated)

        return augmented

### 📈 Data Distribution Strategies

**IID (Independent and Identically Distributed)**:
- Data is randomly shuffled and evenly distributed among clients
- Each client gets a representative sample of the overall data distribution
- Simulates ideal federated learning conditions

**Non-IID (Non-Independent and Identically Distributed)**:
- Uses Dirichlet distribution (α parameter) to create skewed data distributions
- Lower α values create more heterogeneous data across clients
- Simulates real-world federated learning challenges where clients have different data patterns

In [ ]:
class FederatedPostureDataModule(pl.LightningDataModule):
    def __init__(
            self,
            csv_file: str,
            num_clients: int = 5,
            batch_size: int = 32,
            num_workers: int = 4,
            iid: bool = True,
            alpha: float = 0.5,
            augment_data: bool = True,
            augment_factor: float = 2.0,  # How much to increase dataset size
            use_smote: bool = True,
            noise_std: float = 0.05,
            augment_prob: float = 0.5,
    ):
        super().__init__()
        self.csv_file = csv_file
        self.num_clients = num_clients
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.iid = iid
        self.alpha = alpha
        self.augment_data = augment_data
        self.augment_factor = augment_factor
        self.use_smote = use_smote
        self.noise_std = noise_std
        self.augment_prob = augment_prob

        self.scaler = StandardScaler()
        self.client_datasets = []
        self.test_ds = None

    def _generate_synthetic_samples(self, X, y):
        """Generate synthetic samples using multiple techniques"""
        synthetic_X, synthetic_y = [], []

        if self.use_smote and len(np.unique(y)) > 1:
            # Use SMOTE for balanced synthetic generation
            smote = SMOTE(random_state=42, k_neighbors=min(3, len(X) - 1))
            try:
                X_smote, y_smote = smote.fit_resample(X, y)
                # Only keep the synthetic samples (SMOTE returns original + synthetic)
                n_original = len(X)
                synthetic_X.append(X_smote[n_original:])
                synthetic_y.append(y_smote[n_original:])
            except ValueError:
                print("SMOTE failed, falling back to noise-based augmentation")

        # Noise-based augmentation
        n_synthetic = int(len(X) * (self.augment_factor - 1))
        if n_synthetic > 0:
            # Randomly select samples to augment
            indices = np.random.choice(len(X), size=n_synthetic, replace=True)

            for idx in indices:
                original_sample = X[idx].copy()
                original_label = y[idx]

                # Add controlled noise
                noise = np.random.normal(0, self.noise_std, size=original_sample.shape)
                synthetic_sample = original_sample + noise

                # Add some physiologically reasonable variations
                # For posture data, small angle changes are realistic
                angle_variation = np.random.normal(0, 0.02, size=original_sample.shape)
                synthetic_sample += angle_variation

                synthetic_X.append(synthetic_sample)
                synthetic_y.append(original_label)

        if synthetic_X:
            if len(synthetic_X) == 1:
                return synthetic_X[0], np.array(synthetic_y)
            else:
                return np.vstack(synthetic_X), np.hstack(synthetic_y)
        else:
            return np.array([]), np.array([])

    def _add_regularization_noise(self, X, y):
        """Add regularization through controlled data corruption"""
        noisy_X = []
        noisy_y = []

        # Create "hard" examples by adding more noise to some samples
        n_hard_examples = int(len(X) * 0.1)  # 10% hard examples
        hard_indices = np.random.choice(len(X), size=n_hard_examples, replace=False)

        for idx in hard_indices:
            sample = X[idx].copy()
            label = y[idx]

            # Add stronger noise to create challenging examples
            strong_noise = np.random.normal(0, self.noise_std * 2, size=sample.shape)
            noisy_sample = sample + strong_noise

            noisy_X.append(noisy_sample)
            noisy_y.append(label)

        return np.array(noisy_X), np.array(noisy_y)

    def setup(self, stage: str) -> None:
        # Load original data
        df = pd.read_csv(self.csv_file)
        X_original = df[
            ["neck_angle", "torso_angle", "shoulders_offset", "relative_neck_angle"]
        ].values
        y_original = df["good_posture"].astype(int).values

        print(f"Original dataset size: {len(X_original)}")

        # Apply augmentation if enabled
        if self.augment_data and stage == "fit":
            # Generate synthetic samples
            X_synthetic, y_synthetic = self._generate_synthetic_samples(
                X_original, y_original
            )

            # Add regularization noise
            X_noisy, y_noisy = self._add_regularization_noise(X_original, y_original)

            # Combine all data
            X_combined = [X_original]
            y_combined = [y_original]

            if len(X_synthetic) > 0:
                X_combined.append(X_synthetic)
                y_combined.append(y_synthetic)
                print(f"Added {len(X_synthetic)} synthetic samples")

            if len(X_noisy) > 0:
                X_combined.append(X_noisy)
                y_combined.append(y_noisy)
                print(f"Added {len(X_noisy)} noisy regularization samples")

            X = np.vstack(X_combined)
            y = np.hstack(y_combined)

            print(
                f"Augmented dataset size: {len(X)} (factor: {len(X) / len(X_original):.2f}x)"
            )
        else:
            X, y = X_original, y_original

        # Scale features
        X_scaled = self.scaler.fit_transform(X)

        if stage == "fit":
            # Partition data across clients
            if self.iid:
                client_indices = self._partition_data_iid(len(X_scaled))
            else:
                client_indices = self._partition_data_non_iid(y)

            # Create client datasets with augmentation
            self.client_datasets = []
            for indices in client_indices:
                if len(indices) > 0:
                    client_X = X_scaled[indices]
                    client_y = y[indices]

                    # Create augmented dataset for this client
                    client_dataset = AugmentedPostureDataset(
                        client_X,
                        client_y,
                        augment=self.augment_data,
                        noise_std=self.noise_std,
                        augment_prob=self.augment_prob,
                    )
                    self.client_datasets.append(client_dataset)
                else:
                    # Fallback for empty client
                    self.client_datasets.append(
                        AugmentedPostureDataset(X_scaled[:1], y[:1], augment=False)
                    )

        if stage == "test":
            # Don't augment test data
            self.test_ds = TensorDataset(
                torch.FloatTensor(X_scaled), torch.LongTensor(y)
            )

    def _partition_data_iid(self, dataset_size: int) -> List[List[int]]:
        """Partition data indices in IID manner"""
        indices = np.random.permutation(dataset_size)
        client_indices = np.array_split(indices, self.num_clients)
        return [idx.tolist() for idx in client_indices]

    def _partition_data_non_iid(self, labels: np.ndarray) -> List[List[int]]:
        """Partition data indices in non-IID manner using Dirichlet distribution"""
        num_classes = len(np.unique(labels))
        client_indices = [[] for _ in range(self.num_clients)]

        for class_id in range(num_classes):
            class_indices = np.where(labels == class_id)[0]
            np.random.shuffle(class_indices)

            proportions = np.random.dirichlet(np.repeat(self.alpha, self.num_clients))
            proportions = np.cumsum(proportions)

            start_idx = 0
            for client_id in range(self.num_clients):
                end_idx = int(proportions[client_id] * len(class_indices))
                client_indices[client_id].extend(class_indices[start_idx:end_idx])
                start_idx = end_idx

        for client_id in range(self.num_clients):
            np.random.shuffle(client_indices[client_id])

        return client_indices

    def get_client_dataloader(self, client_id: int) -> DataLoader:
        """Get dataloader for specific client"""
        if client_id >= len(self.client_datasets):
            raise ValueError(
                f"Client {client_id} does not exist. Only {len(self.client_datasets)} clients available."
            )

        return DataLoader(
            self.client_datasets[client_id],
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True,
            persistent_workers=True if self.num_workers > 0 else False,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            persistent_workers=True if self.num_workers > 0 else False,
        )

    def get_client_data_info(self) -> dict:
        """Get information about data distribution across clients"""
        info = {}
        for i, dataset in enumerate(self.client_datasets):
            # Count labels in the dataset
            labels = [dataset.labels[j].item() for j in range(len(dataset))]
            unique, counts = np.unique(labels, return_counts=True)
            info[f"client_{i}"] = {
                "total_samples": len(dataset),
                "class_distribution": dict(zip(unique.tolist(), counts.tolist())),
            }
        return info

## 👥 Federated Client Implementation


### 🔄 Client Workflow

Each `FederatedClient` operates in the following cycle:

1. **Model Update**: Receives global model weights from server
2. **Local Training**: Trains on local data for specified epochs
3. **Weight Extraction**: Returns updated model weights and dataset size
4. **Local Evaluation**: Measures performance on local data

The client never shares raw data - only model weights are exchanged, preserving privacy.

In [ ]:
import copy
from typing import Dict, Tuple

import torch
from torch import nn
from torch.utils.data import DataLoader


class FederatedClient:
    def __init__(self, client_id: int, model: nn.Module, dataloader: DataLoader):
        self.client_id = client_id
        self.model = copy.deepcopy(model)
        self.dataloader = dataloader
        self.dataset_size = len(dataloader.dataset)

    def update_model(self, global_weights: Dict):
        """Update local model with global weights"""
        self.model.load_state_dict(global_weights)

    def local_train(
        self, epochs: int = 5, learning_rate: float = 0.001
    ) -> Tuple[Dict, int]:
        """Train model locally and return updated weights"""
        self.model.train()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            for batch in self.dataloader:
                inputs, labels = batch

                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

        return copy.deepcopy(self.model.state_dict()), self.dataset_size

    def evaluate(self) -> Dict:
        """Evaluate local model"""
        self.model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        criterion = nn.CrossEntropyLoss()

        with torch.no_grad():
            for batch in self.dataloader:
                inputs, labels = batch
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total if total > 0 else 0
        avg_loss = total_loss / len(self.dataloader) if len(self.dataloader) > 0 else 0

        return {"accuracy": accuracy, "loss": avg_loss}


## 🌐 Federated Server Implementation


### ⚖️ FedAvg Algorithm Implementation

The server implements the **Federated Averaging (FedAvg)** algorithm:

```
w_global = Σ(n_k / n_total) * w_k
```

Where:
- `w_global`: Global model weights
- `n_k`: Number of samples at client k
- `n_total`: Total samples across all clients
- `w_k`: Local model weights from client k

This weighted averaging ensures that clients with more data have proportionally more influence on the global model.


In [ ]:
import copy
from typing import List, Dict

import numpy as np
import torch
import torch.nn as nn
from lightning.pytorch.loggers import TensorBoardLogger
from torch.utils.tensorboard import SummaryWriter


class FederatedServer:
    def __init__(
        self,
        model: nn.Module,
        num_clients: int,
        client_fraction: float = 1.0,
        logger: TensorBoardLogger = None,
    ):
        self.global_model = model
        self.num_clients = num_clients
        self.client_fraction = client_fraction
        self.round_history = []

        # TensorBoard integration
        self.logger = logger
        if self.logger:
            self.writer = self.logger.experiment
        else:
            # Fallback to direct SummaryWriter
            self.writer = SummaryWriter(log_dir="logs/federated_learning")

    def select_clients(self, round_num: int) -> List[int]:
        """Select subset of clients for this round"""
        num_selected = max(1, int(self.client_fraction * self.num_clients))
        np.random.seed(round_num)
        selected_clients = np.random.choice(
            range(self.num_clients), size=num_selected, replace=False
        ).tolist()
        return selected_clients

    def aggregate_weights(
        self, client_weights: List[Dict], client_sizes: List[int]
    ) -> Dict:
        """Implement FedAvg algorithm"""
        total_samples = sum(client_sizes)
        aggregated_weights = copy.deepcopy(client_weights[0])

        for key in aggregated_weights.keys():
            aggregated_weights[key] = torch.zeros_like(aggregated_weights[key])

            for i, client_weight in enumerate(client_weights):
                weight = client_sizes[i] / total_samples
                aggregated_weights[key] += weight * client_weight[key]

        return aggregated_weights

    def update_global_model(self, aggregated_weights: Dict):
        """Update global model with aggregated weights"""
        self.global_model.load_state_dict(aggregated_weights)

    def get_global_weights(self) -> Dict:
        """Get current global model weights"""
        return copy.deepcopy(self.global_model.state_dict())

    def evaluate_global_model(self, test_dataloader, round_num: int) -> Dict:
        """Evaluate global model on test data and log to TensorBoard"""
        self.global_model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        criterion = nn.CrossEntropyLoss()

        with torch.no_grad():
            for batch in test_dataloader:
                inputs, labels = batch
                outputs = self.global_model(inputs)
                loss = criterion(outputs, labels)

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        avg_loss = total_loss / len(test_dataloader)

        # Log to TensorBoard
        self.writer.add_scalar("Global/Accuracy", accuracy, round_num)
        self.writer.add_scalar("Global/Loss", avg_loss, round_num)

        return {"accuracy": accuracy, "loss": avg_loss}

    def log_client_metrics(
        self,
        client_accuracies: List[float],
        client_losses: List[float],
        selected_clients: List[int],
        round_num: int,
    ):
        """Log individual client metrics to TensorBoard"""
        avg_client_acc = np.mean(client_accuracies)
        avg_client_loss = np.mean(client_losses)

        # Log average client metrics
        self.writer.add_scalar("Clients/Average_Accuracy", avg_client_acc, round_num)
        self.writer.add_scalar("Clients/Average_Loss", avg_client_loss, round_num)

        # Log individual client metrics
        for i, (client_id, acc, loss) in enumerate(
            zip(selected_clients, client_accuracies, client_losses)
        ):
            self.writer.add_scalar(f"Client_{client_id}/Accuracy", acc, round_num)
            self.writer.add_scalar(f"Client_{client_id}/Loss", loss, round_num)

    def log_model_weights(self, round_num: int):
        """Log model weight histograms to TensorBoard"""
        for name, param in self.global_model.named_parameters():
            self.writer.add_histogram(f"Global_Weights/{name}", param, round_num)


## 🎯 Federated Trainer - Orchestrating Everything


The `FederatedTrainer` coordinates the entire federated learning process:

1. **Initialization**: Sets up data module, model, server, and clients
2. **Client Selection**: Randomly selects clients for each round
3. **Local Training**: Selected clients train on their local data
4. **Weight Aggregation**: Server aggregates client weights using FedAvg
5. **Global Evaluation**: Tests the global model on centralized test data
6. **Logging**: Records metrics to TensorBoard for visualization
7. **Checkpointing**: Saves model states periodically


In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
from lightning.pytorch.loggers import TensorBoardLogger

from client import FederatedClient
from datamodule import FederatedPostureDataModule
from model import PostureMLP
from server import FederatedServer


class FederatedTrainer:
    def __init__(
        self,
        csv_file: str,
        num_clients: int = 5,
        num_rounds: int = 50,
        local_epochs: int = 5,
        client_fraction: float = 1.0,
        learning_rate: float = 0.001,
        batch_size: int = 32,
        iid: bool = True,
        save_dir: str = "logs",
        experiment_name: str = "federated_posture",
    ):
        self.csv_file = csv_file
        self.num_clients = num_clients
        self.num_rounds = num_rounds
        self.local_epochs = local_epochs
        self.client_fraction = client_fraction
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.iid = iid

        # Setup TensorBoard logger (Lightning style)
        self.logger = TensorBoardLogger(
            save_dir=save_dir,
            name=experiment_name,
            version=None,  # Auto-increment version
        )

        # Create checkpoint directory
        self.checkpoint_dir = os.path.join(self.logger.log_dir, "checkpoints")
        os.makedirs(self.checkpoint_dir, exist_ok=True)

        # Initialize components
        self.datamodule = FederatedPostureDataModule(
            csv_file=csv_file,
            num_clients=num_clients,
            batch_size=batch_size,
            iid=iid,
            augment_data=True,  # Enable augmentation
            augment_factor=10.0,  # Double the dataset size
            use_smote=True,  # Use SMOTE for balanced generation
            noise_std=2,  # Noise level
            augment_prob=0.9  # 50% chance of augmentation per sample
        )

        self.global_model = PostureMLP()
        self.server = FederatedServer(
            self.global_model, num_clients, client_fraction, logger=self.logger
        )

        # Training history
        self.history = {
            "round": [],
            "global_accuracy": [],
            "global_loss": [],
            "client_accuracies": [],
        }

    def setup_clients(self):
        """Setup federated data and create clients"""
        self.datamodule.setup("fit")
        self.datamodule.setup("test")

        # Create clients
        self.clients = []
        for i in range(self.num_clients):
            client_dataloader = self.datamodule.get_client_dataloader(i)
            client = FederatedClient(i, self.global_model, client_dataloader)
            self.clients.append(client)

        # Log data distribution info to TensorBoard
        data_info = self.datamodule.get_client_data_info()

        # Create data distribution visualization
        fig, ax = plt.subplots(figsize=(12, 6))
        client_ids = []
        good_posture_counts = []
        bad_posture_counts = []

        for client_id, info in data_info.items():
            client_ids.append(client_id.replace("client_", "Client "))
            good_posture_counts.append(info["class_distribution"].get(1, 0))
            bad_posture_counts.append(info["class_distribution"].get(0, 0))

        x = range(len(client_ids))
        width = 0.35

        ax.bar(
            [i - width / 2 for i in x],
            bad_posture_counts,
            width,
            label="Bad Posture",
            color="red",
            alpha=0.7,
        )
        ax.bar(
            [i + width / 2 for i in x],
            good_posture_counts,
            width,
            label="Good Posture",
            color="green",
            alpha=0.7,
        )

        ax.set_xlabel("Clients")
        ax.set_ylabel("Number of Samples")
        ax.set_title(
            f'Data Distribution Across Clients ({"IID" if self.iid else "Non-IID"})'
        )
        ax.set_xticks(x)
        ax.set_xticklabels(client_ids)
        ax.legend()
        ax.grid(True, alpha=0.3)

        plt.tight_layout()
        self.logger.experiment.add_figure("Data_Distribution", fig, 0)
        plt.close(fig)

        print("Data distribution across clients:")
        for client_id, info in data_info.items():
            print(f"{client_id}: {info}")

    def train_federated(self):
        """Main federated training loop with TensorBoard logging"""
        print(f"Starting Federated Learning with {self.num_clients} clients")
        print(f"Data distribution: {'IID' if self.iid else 'Non-IID'}")
        print(f"TensorBoard logs will be saved to: {self.logger.log_dir}")
        print("-" * 60)

        test_dataloader = self.datamodule.test_dataloader()

        # Log hyperparameters
        hparams = {
            "num_clients": self.num_clients,
            "num_rounds": self.num_rounds,
            "local_epochs": self.local_epochs,
            "client_fraction": self.client_fraction,
            "learning_rate": self.learning_rate,
            "batch_size": self.batch_size,
            "iid": self.iid,
        }
        self.logger.log_hyperparams(hparams)

        for round_num in range(self.num_rounds):
            print(f"Round {round_num + 1}/{self.num_rounds}")

            # Select clients for this round
            selected_clients = self.server.select_clients(round_num)
            print(f"Selected clients: {selected_clients}")

            # Collect client updates
            client_weights = []
            client_sizes = []
            client_accuracies = []
            client_losses = []

            for client_id in selected_clients:
                client = self.clients[client_id]

                # Update client with global model
                client.update_model(self.server.get_global_weights())

                # Local training
                weights, size = client.local_train(
                    epochs=self.local_epochs, learning_rate=self.learning_rate
                )

                # Evaluate client
                client_eval = client.evaluate()

                client_weights.append(weights)
                client_sizes.append(size)
                client_accuracies.append(client_eval["accuracy"])
                client_losses.append(client_eval["loss"])

                print(
                    f"  Client {client_id}: Accuracy = {client_eval['accuracy']:.2f}%"
                )

            # Log client metrics to TensorBoard
            self.server.log_client_metrics(
                client_accuracies, client_losses, selected_clients, round_num
            )

            # Aggregate weights using FedAvg
            aggregated_weights = self.server.aggregate_weights(
                client_weights, client_sizes
            )
            self.server.update_global_model(aggregated_weights)

            # Log model weights every 5 rounds
            if round_num % 5 == 0:
                self.server.log_model_weights(round_num)

            # Evaluate global model
            global_eval = self.server.evaluate_global_model(test_dataloader, round_num)

            # Store history
            self.history["round"].append(round_num + 1)
            self.history["global_accuracy"].append(global_eval["accuracy"])
            self.history["global_loss"].append(global_eval["loss"])
            self.history["client_accuracies"].append(np.mean(client_accuracies))

            # Save checkpoint
            if round_num % 10 == 0 or round_num == self.num_rounds - 1:
                checkpoint_path = os.path.join(
                    self.logger.log_dir,
                    "checkpoints",
                    f"federated-round-{round_num:02d}-acc-{global_eval['accuracy']:.2f}.ckpt",
                )
                torch.save(
                    {
                        "round": round_num,
                        "model_state_dict": self.global_model.state_dict(),
                        "global_accuracy": global_eval["accuracy"],
                        "global_loss": global_eval["loss"],
                        "hyperparameters": hparams,
                    },
                    checkpoint_path,
                )

            print(
                f"  Global Model: Accuracy = {global_eval['accuracy']:.2f}%, Loss = {global_eval['loss']:.4f}"
            )
            print(f"  Average Client Accuracy = {np.mean(client_accuracies):.2f}%")
            print("-" * 60)

        print(f"\nFederated Training completed!")
        print(f"TensorBoard logs saved to: {self.logger.log_dir}")
        print(f"To view results, run: tensorboard --logdir={self.logger.save_dir}")


## 🚀 Main Execution Script


### Hyperparameters
- **Clients**: 5 participants
- **Rounds**: 30 federated learning rounds
- **Local Epochs**: 5 epochs per client per round
- **Batch Size**: 64 samples
- **Learning Rate**: 0.001 (Adam optimizer)
- **Augmentation Factor**: 10x dataset increase
- **Noise Standard Deviation**: 2.0
- **Augmentation Probability**: 90%

### Experiments
1. **IID Scenario**: Even data distribution across clients
2. **Non-IID Scenario**: Skewed data distribution simulating real-world conditions


In [ ]:
# federated/federated_main.py
import os
import sys

import numpy as np
import torch

from trainer import FederatedTrainer

# Add parent directory to path to import model
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

BATCH_SIZE = 64
NUM_WORKERS = 15
NUM_CLIENTS = 5
NUM_ROUNDS = 30
LOCAL_EPOCHS = 5

if __name__ == "__main__":
    torch.set_float32_matmul_precision("high")

    # IID Federated Learning
    print("=" * 80)
    print("FEDERATED LEARNING - IID")
    print("=" * 80)

    fed_trainer_iid = FederatedTrainer(
        csv_file="../datasets/train.csv",  # Adjust path as needed
        num_clients=NUM_CLIENTS,
        num_rounds=NUM_ROUNDS,
        local_epochs=LOCAL_EPOCHS,
        client_fraction=1.0,
        learning_rate=0.001,
        batch_size=BATCH_SIZE,
        iid=True,
        save_dir="logs",
        experiment_name="federated_posture_iid",
    )

    fed_trainer_iid.setup_clients()
    fed_trainer_iid.train_federated()

    # Non-IID Federated Learning
    print("\n" + "=" * 80)
    print("FEDERATED LEARNING - NON-IID")
    print("=" * 80)

    fed_trainer_non_iid = FederatedTrainer(
        csv_file="../datasets/train.csv",  # Adjust path as needed
        num_clients=NUM_CLIENTS,
        num_rounds=NUM_ROUNDS,
        local_epochs=LOCAL_EPOCHS,
        client_fraction=1.0,
        learning_rate=0.001,
        batch_size=BATCH_SIZE,
        iid=False,
        save_dir="logs",
        experiment_name="federated_posture_non_iid",
    )

    fed_trainer_non_iid.setup_clients()
    fed_trainer_non_iid.train_federated()

    print(f"\nAll experiments completed!")
    print(f"To view all results, run: tensorboard --logdir=logs")


## 🎯 Key Features & Innovations

### 🔒 Privacy Preservation
- **No Raw Data Sharing**: Only model weights are exchanged
- **Local Training**: Each client trains exclusively on their own data
- **Differential Privacy Ready**: Framework supports adding noise to weights

### 🎲 Advanced Data Augmentation
- **SMOTE Integration**: Synthetic minority oversampling for balanced datasets
- **Physiologically Realistic Noise**: Correlated noise between related features
- **Real-time Augmentation**: On-the-fly data augmentation during training
- **Regularization Techniques**: Hard example generation for improved robustness

### 📈 Comprehensive Logging
- **TensorBoard Integration**: Real-time visualization of training progress
- **Model Checkpointing**: Automatic saving of best models
- **Data Distribution Visualization**: Charts showing client data heterogeneity
- **Confusion Matrix Tracking**: Classification performance analysis

### 🔄 Flexible Architecture
- **IID/Non-IID Support**: Handles both ideal and realistic data distributions
- **Configurable Client Selection**: Partial client participation per round
- **Scalable Design**: Easy to extend to more clients or different architectures

---

## 🎉 Expected Outcomes

### Performance Metrics
- **Global Model Accuracy**: 85-95% on test data
- **Convergence Speed**: Typically converges within 20-30 rounds
- **Client Fairness**: Balanced performance across all clients

### Insights
- **IID vs Non-IID**: IID typically achieves better performance and faster convergence
- **Data Augmentation Impact**: 10x augmentation significantly improves robustness
- **Communication Efficiency**: Only weights transmitted, not raw data

---

## 🔧 Usage Instructions

1. **Install Dependencies**:
   ```bash
   pip install torch lightning tensorboard scikit-learn imbalanced-learn pandas numpy matplotlib seaborn
   ```

2. **Prepare Data**: Ensure your CSV file has columns: `neck_angle`, `torso_angle`, `shoulders_offset`, `relative_neck_angle`, `good_posture`

3. **Run Training**:
   ```bash
   python main.py
   ```

4. **Monitor Progress**:
   ```bash
   tensorboard --logdir=logs
   ```

5. **View Results**: Open http://localhost:6006 in your browser

---

## 🎯 Conclusion

This federated learning system demonstrates how to:
- Train models across distributed clients while preserving privacy
- Handle both ideal (IID) and realistic (Non-IID) data distributions
- Implement sophisticated data augmentation for small datasets
- Create comprehensive logging and visualization systems
- Build scalable, production-ready federated learning pipelines

The system is particularly well-suited for healthcare applications where data privacy is paramount, such as posture monitoring, activity recognition, or medical diagnosis systems.